# Spark Exercises

In [1]:
import numpy as np
import pandas as pd

import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

## 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be `language`
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe